#### sqlite3 API   
1. sqlite3.**connect**(database [,timeout, other optional arguments])  
    :memory: option
2. connection.**cursor**([cursorClass])  
3. cursor.**execute**(sql [, optional parameters])   
    E.g., cursor.execute("insert into people values (?, ?)", (name, age))   
4. connection.execute(sql [, optional parameters])    
    creates its own intermediate cursor.  
5. cursor.**executemany**(sql, seq_of_parameters)   
    connection.executemany(sql[, parameters])   
6. cursor.**executescript**(sql_script)   
    connection.executescript(sql_script)  
7. connection.**total_changes**()  
    total rows modified, inserted, or deleted since the database connection was opened.  
8. connection.**commit**()   
9. connection.**rollback**()  
    roll back changes to the database since the last call to commit().
10. connection.**close**()  
    closing without commit(), **changes will be lost**.
11. cursor.**fetchone**()  
    fetch the next row of a query result set.
12. cursor.**fetchmany**([size = cursor.arraysize])  
    tries to fetch as many rows as indicated by the size parameter.
13. cursor.**fetchall**()  
    fetch all the remaining rows

##### Imposter table   
is attached to b-tree as an index.   
It allows the content of an index to be queried or modified as if the index were an ordinary table.  

Imposter tables are intended for power analysis and debugging only.  
They are for experts only.  
Improper use of imposter tables can cause index corruption.   

The root page number for any index or table can be found by querying the "rootpage" column of the sqlite_schema table.  

In [1]:
import pandas as pd
import sqlite3 as lite
import re
import numpy 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##### Database structure and contents    
```
sqlite_master (
    type text,         -- eneity type
    name text,         -- entity name 
    tbl_name text,     -- associated table
    rootpage integer,  -- page in the B-Tree of data 
    sql text           -- table schema 
);
```

In [3]:
with lite.connect('../Databases/BTech22.sqlite') as con:
    query = "SELECT * FROM sqlite_master"
    result = (list(con.execute(query).fetchall()))  

print("\ntype,   name,  table-name, schema\n")    
for r in result:
    print(f"{r[:3]}\n{r[4]}\n")


type,   name,  table-name, schema

('table', 'Students', 'Students')
CREATE TABLE "Students" (
"USN" TEXT,
  "Name" TEXT,
  "Email" TEXT,
  "Section" TEXT,
  "Sex" TEXT,
  "DoB" TEXT,
  "Height_cm" INTEGER,
  "Weight_Kg" INTEGER,
  "Home_State" TEXT,
  "Home_Town" TEXT,
  "Mother_Tongue" TEXT,
  "Elective" TEXT,
  "Clubs" TEXT
)



---
#### Basic Queries  
1. On 25 Apr 2023, CEO-KA released AC-wise count of voters. /Databases/KA_2305_VoterCounts.csv is extracted from the PDF document.  
2. Create a database named 'Elections_2023.sqlite' and import the voter counts document.  

##### Create table  

In [4]:
sql = """CREATE TABLE IF NOT EXISTS VoterCounts 
         (District Text, 
          AC_Num Integer, 
          AC_Name Text, 
          Male Integer, 
          Female Integer, 
          Others Integer, 
          Total Integer
         );
"""
with lite.connect('../Databases/Elections_2023.sqlite') as con:
    con.execute(sql)
    con.commit()

##### Verify  

In [5]:
con =  lite.connect('../Databases/Elections_2023.sqlite')
sql = "SELECT name from sqlite_master WHERE type='table'"
list(con.execute(sql))  

[('VoterCounts',)]

In [6]:
sql = "Pragma table_info(VoterCounts)"
list(con.execute(sql)) 

[(0, 'District', 'TEXT', 0, None, 0),
 (1, 'AC_Num', 'INTEGER', 0, None, 0),
 (2, 'AC_Name', 'TEXT', 0, None, 0),
 (3, 'Male', 'INTEGER', 0, None, 0),
 (4, 'Female', 'INTEGER', 0, None, 0),
 (5, 'Others', 'INTEGER', 0, None, 0),
 (6, 'Total', 'INTEGER', 0, None, 0)]

In [13]:
with lite.connect('../Databases/Elections_2023.sqlite') as con:
    sql = "SELECT COUNT(*) FROM VoterCounts"
    # print(f"Number of Constituencies: {con.execute(sql).fetchone()[0]}")   
    print(f"Number of Constituencies: {con.execute(sql).fetchone()}")   

Number of Constituencies: (224,)


##### Delete all rows   
- commit after the changes   
- vacuum to release unused space   
- TRUNCATE in some other RDBMS   

**check** the row count 

In [9]:
with lite.connect('../Databases/Elections_2023.sqlite') as con:
    con.execute("DELETE FROM VoterCounts")
    con.commit()
    con.execute("VACUUM")

In [11]:
import csv

file = open('../Databases/KA_2305_VoterCounts.csv')
contents = list(csv.reader(file))
print(f"\nrows and columns: {numpy.shape(contents)}\nSample:")
list(contents)[:3]


rows and columns: (224, 7)
Sample:


[['BELGAUM', '1', 'Nippani', '113856', '111823', '9', '225688'],
 ['BELGAUM', '2', 'Chikkodi-Sadalga', '112437', '111388', '9', '223834'],
 ['BELGAUM', '3', 'Athani', '116641', '111499', '4', '228144']]

In [12]:
insert_records = """INSERT INTO 
                    VoterCounts(District, AC_Num, AC_Name, Male, Female, Others, Total) 
                    VALUES(?, ?, ?, ?, ?, ?, ?)"""

with lite.connect('../Databases/Elections_2023.sqlite') as con:
    con.executemany(insert_records, contents)
    con.commit()

In [14]:
con.close()

---  
### Queries    
##### Row counts   

In [35]:
sql = "SELECT COUNT(*) FROM VoterCounts"
print(f"Total Rows: {list(con.execute(sql))[0][0]}")

Total Rows: 224
